This code was designed by me but AI helped me a lot to implement the BERTopic . I used the first labs for the preprocesing, used ChatGPT and Copilot for debugging, identifying errors, and improving efficiency. Also AI helped me to mae the topic model more efficient suggesting me libaries

In [ ]:
!pip install bertopic

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import random, numpy as np, torch
from tqdm import tqdm


In [ ]:
# Selecting a random seed in order to have consistent reuslts for the Topic Model
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
df = pd.read_csv(r"/content/drive/MyDrive/ni_06.csv")

In [ ]:
print(df.columns)
print(df.shape)


Index(['speechID', 'memberID', 'partyID', 'constID', 'title', 'date',
       'member_name', 'party_name', 'const_name', 'speech', 'speechnumber',
       'paragraphnumber', 'sentencenumber', 'parliament', 'iso3country',
       'speaker_uri', 'eu', 'policyarea', 'cmp_party', 'year', 'word_count',
       'combined_text', 'ni_score', 'five_year_period', 'ni_similarity'],
      dtype='object')
(4619, 25)


# === PREPROCESS ===

In [ ]:
import re
from nltk.corpus import stopwords
import nltk
import spacy

# Download stopwords once
nltk.download('stopwords')

# Load English stopwords
stop_words_en = set(stopwords.words('english'))

# Add your custom Irish stopwords manually
irish_stopwords = set([
    'agus', 'go', 'le', 'ní', 'an', 'na', 'ar', 'a', 'do', 'is', 'bhí', 'sé',
    'seo', 'bhfuil', 'tá', 'ó', 'raibh', 'mar', 'dom', 'chuig', 'ón', 'nach', 'leis'
])

# Combine both
stop_words = stop_words_en.union(irish_stopwords)

# Load spacy model
nlp = spacy.load("en_core_web_sm")  # You can switch if you're handling Irish separately

# Text cleaning
def clean_text(text):
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespace
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"\d+", "", text)  # Remove numbers
    return text.strip()

# Stopword removal
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word.lower() not in stop_words])

# Lemmatization
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.lemma_.isalpha()])

# All together
def preprocess(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tqdm.pandas()
df['clean_speech'] = df['speech'].progress_apply(preprocess)


100%|██████████| 4619/4619 [04:25<00:00, 17.38it/s] 


# === EMBEDDINGS ===

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

clean_sentences = df['clean_speech'].tolist()
embeddings = model.encode(clean_sentences, show_progress_bar=True)

df['embeddings'] = embeddings.tolist()

Batches:   0%|          | 0/145 [00:00<?, ?it/s]


# === BERTopic SETUP ===

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import umap
import hdbscan

umap_model = umap.UMAP(
    n_neighbors=15, # distance between topics
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=30,  # minimum speeches per topic
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# Modify the model to calculate probabilities
topic_model = BERTopic(
    embedding_model=model,  # Insert your embedding model here (e.g., BERT)
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,  # Set a minimum size to avoid very small clusters
    nr_topics="auto"    # Let the model decide automatically
)


In [ ]:

docs = df['clean_speech'].tolist()

# Apply the BERTopic model
topics, probs = topic_model.fit_transform(docs)

#  Get document information (includes the probability of the main topic)
doc_info = topic_model.get_document_info(docs)

# Create a DataFrame with the probabilities of all topics
probs_df = pd.DataFrame(probs, columns=[f"topic_{i}_prob" for i in range(probs.shape[1])])

# Add the main topic assigned by BERTopic
probs_df["main_topic"] = topics

#  Merge the original DataFrame with the probabilities DataFrame
df_with_probabilities = pd.concat([df.reset_index(drop=True), probs_df], axis=1)


df_with_probabilities.to_excel("/content/drive/MyDrive/discorso_con_topic_prob.xlsx", index=False)
print("✅ Excel file successfully saved with all topic probabilities!")


2025-04-14 11:35:47,586 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/145 [00:00<?, ?it/s]

2025-04-14 11:36:57,769 - BERTopic - Embedding - Completed ✓
2025-04-14 11:36:57,770 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-14 11:37:28,902 - BERTopic - Dimensionality - Completed ✓
2025-04-14 11:37:28,903 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-14 11:37:29,314 - BERTopic - Cluster - Completed ✓
2025-04-14 11:37:29,315 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-04-14 11:37:34,270 - BERTopic - Representation - Completed ✓
2025-04-14 11:37:34,291 - BERTopic - Topic reduction - Reducing number of topics
2025-04-14 11:37:34,304 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-14 11:37:39,818 - BERTopic - Representation - Completed ✓
2025-04-14 11:37:39,841 - BERTopic - Topic reduction - Reduced number of topics from 19 to 19


✅ File Excel salvato correttamente con tutte le probabilità dei topic!


In [ ]:
df_with_probabilities.columns

Index(['speechID', 'memberID', 'partyID', 'constID', 'title', 'date',
       'member_name', 'party_name', 'const_name', 'speech', 'speechnumber',
       'paragraphnumber', 'sentencenumber', 'parliament', 'iso3country',
       'speaker_uri', 'eu', 'policyarea', 'cmp_party', 'year', 'word_count',
       'combined_text', 'ni_score', 'five_year_period', 'ni_similarity',
       'clean_speech', 'embeddings', 'topic_0_prob', 'topic_1_prob',
       'topic_2_prob', 'topic_3_prob', 'topic_4_prob', 'topic_5_prob',
       'topic_6_prob', 'topic_7_prob', 'topic_8_prob', 'topic_9_prob',
       'topic_10_prob', 'topic_11_prob', 'topic_12_prob', 'topic_13_prob',
       'topic_14_prob', 'topic_15_prob', 'topic_16_prob', 'topic_17_prob',
       'main_topic'],
      dtype='object')

In [ ]:

topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2231,-1_government_ireland_people_party,"[government, ireland, people, party, northern,...",[fully agree Minister Health cross border road...
1,0,691,0_taoiseach_agree_government_british,"[taoiseach, agree, government, british, party,...",[Taoiseach Government fail duty dealing Britis...
2,1,278,1_ireland_people_northern_say,"[ireland, people, northern, say, north, northe...",[wish deal length Bill observation like make d...
3,2,245,2_agreement_ireland_irish_government,"[agreement, ireland, irish, government, northe...",[fully support amendment name Deputy Vincent B...
4,3,182,3_people_government_say_house,"[people, government, say, house, country, depu...",[Fine Gael Party precipitate motion could well...
5,4,133,4_agreement_party_government_northern,"[agreement, party, government, northern, north...",[propose take Questions Nos inclusive together...
6,5,113,5_offence_act_court_extradition,"[offence, act, court, extradition, law, people...",[five people already charge connection kidnapp...
7,6,113,6_ceasefire_sinn_sinn féin_féin,"[ceasefire, sinn, sinn féin, féin, ira, talk, ...",[welcome opportunity afford statement report H...
8,7,90,7_european_treaty_union_europe,"[european, treaty, union, europe, country, com...",[speak Dáil Éireann today Taoiseach party lead...
9,8,82,8_party_issue_agreement_try,"[party, issue, agreement, try, position, make,...",[make number comment reply Second Stage Deputy...


In [ ]:
# Save the BERTopic model
topic_model.save("/content/drive/MyDrive/my_bertopic_model")

print("✅ Model saved successfully!")


2025-04-14 11:49:35,819 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


✅ Model saved successfully!
